# 0 Load dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!nvidia-smi

In [ ]:
PROJECT_path = '/content/drive/MyDrive/IDB_diamond_damage'

In [ ]:
import os
import numpy as np
from tqdm import tqdm
from PIL import Image

In [ ]:
def load_dataset(dataset_path):
  image_list = []
  label_list = []
  tag = 0
  category_names = os.listdir(dataset_path)
  category_nums = len(category_names)
  category_names.sort()
  print(category_names)
  for category in category_names:
    category_path = os.path.join(dataset_path, category)
    file_names = os.listdir(category_path)
    file_nums = len(file_names)
    file_names.sort()  
    for file in tqdm(file_names):
      file_path = os.path.join(category_path, file)
      image = Image.open(file_path)
      img = np.asarray(image,dtype="float32")

      #chose wheather to crop the images, e.g., 1024*1024
      img = img[0:1024, 0:1024]
      
      img = img[:, :, np.newaxis] 
      image_list.append(img)
      label_list.append(tag)
    tag += 1
  return image_list, label_list

In [ ]:
X_set, Y_set = load_dataset(os.path.join(PROJECT_path, 'SEM'))

In [ ]:
CLASS_num = 65
CLASS = ["#01", "#02", "#03", "#04", "#05", "#06", "#07", "#08", "#09", "#10", "#11", "#12", "#13", "#14", "#15", "#16", "#17", "#18", "#19", "#20", "#21", "#22", "#23", "#24", "#25", "#26", "#27", "#28", "#29", "#30", "#31", "#32", "#33", "#34", "#35", "#36", "#37", "#38", "#39", "#40", "#41", "#42", "#43", "#44", "#45", "#46", "#47", "#48", "#49", "#50", "#51", "#52", "#53", "#54", "#55", "#56", "#57", "#58", "#59", "#60", "#61", "#62", "#63", "#64", "#65"]

In [ ]:
save_dir = os.path.join(PROJECT_path, 'saved_models')
if not os.path.isdir(save_dir):
  os.makedirs(save_dir)

# 1 Dataset processing

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
def classification_dataset_process(X_set, Y_set):
  
  # choose the size to convert, e.g., 224*224
  X_set = [cv2.cvtColor(cv2.resize(i, (224, 224)), cv2.COLOR_GRAY2RGB) for i in X_set]

  X_set = np.asarray(X_set)
  X_set = X_set.astype('float32')
  X_set /= 255.0
  Y_set = tf.keras.utils.to_categorical(Y_set, CLASS_num)
  return X_set, Y_set

In [ ]:
X_set, Y_set = classification_dataset_process(X_set, Y_set)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_set,Y_set,random_state = 888)

# 2 VGG16 classification model

## 2.1 [A] model establishment - DIY

In [ ]:
# from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation
# from keras.models import Sequential, Model

In [ ]:
# model = Sequential()

# # Block_1
# model.add(Conv2D(64, (3, 3), padding = 'same', activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv2D(64, (3, 3), padding = 'same', activation='relu'))
# model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# # Block_2
# model.add(Conv2D(128, (3, 3), padding = 'same', activation='relu'))
# model.add(Conv2D(128, (3, 3), padding = 'same', activation='relu'))
# model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# # Block_3
# model.add(Conv2D(256, (3, 3), padding = 'same', activation='relu'))
# model.add(Conv2D(256, (3, 3), padding = 'same', activation='relu'))
# model.add(Conv2D(256, (3, 3), padding = 'same', activation='relu'))
# model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# # Block_4
# model.add(Conv2D(512, (3, 3), padding = 'same', activation='relu'))
# model.add(Conv2D(512, (3, 3), padding = 'same', activation='relu'))
# model.add(Conv2D(512, (3, 3), padding = 'same', activation='relu'))
# model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# # Block_5
# model.add(Conv2D(512, (3, 3), padding = 'same', activation='relu'))
# model.add(Conv2D(512, (3, 3), padding = 'same', activation='relu'))
# model.add(Conv2D(512, (3, 3), padding = 'same', activation='relu'))
# model.add(MaxPooling2D((2, 2), strides=(2, 2)))

# # Block_6
# model.add(Flatten())
# model.add(Dense(4096, activation='relu'))
# model.add(Dense(4096, activation='relu'))
# model.add(Dense(CLASS_num, activation='softmax'))

## 2.1 [B] Model establishment - Import

In [ ]:
from keras import applications
from keras.layers import Flatten, Dense, Dropout, Activation
from keras.models import Sequential, Model

In [ ]:
base_model = applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=X_train.shape[1:])
print(base_model.output)
model = Sequential()
model.add(Flatten(input_shape=base_model.output_shape[1:]))

# add the rest layers
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(CLASS_num, activation='softmax'))
model=Model(inputs=base_model.input, outputs=model(base_model.output))

In [ ]:
# transfer learning or not
for layer in base_model.layers:
  layer.trainable=False

## 2.2 Model compiling

In [ ]:
from keras.models import Model

In [ ]:
# choose approriate optimizer, loss function
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6), 
       loss='categorical_crossentropy', 
       metrics=['accuracy'])

## 2.3 Model training



In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# set hyper-parameters
epochs = 100
batch_size = 10

# whether or not make data augmentation 
data_augmentation = True

# whether or not apply early stopping
early_stopping = False
if not early_stopping:
  callbacks = None
else:
  callbacks = [EarlyStopping(monitor='val_loss', patience=10)]

In [ ]:
if not data_augmentation:
  print('Not using data augmentation.')
  history = model.fit(X_train, Y_train, 
             batch_size=batch_size, 
             epochs=epochs, 
             validation_data=(X_test, Y_test), 
             shuffle=True, 
             callbacks=callbacks)
else:
  print('Using real-time data augmentation.')
  datagen = ImageDataGenerator(featurewise_center=False,  
                  samplewise_center=False,  
                  featurewise_std_normalization=False,  
                  samplewise_std_normalization=False, 
                  zca_whitening=False, 
                  zca_epsilon=1e-06, 
                  rotation_range=0, 
                  width_shift_range=0.1,
                  height_shift_range=0.1,
                  shear_range=0., 
                  zoom_range=0.,
                  channel_shift_range=0., 
                  fill_mode='nearest',
                  cval=0., 
                  horizontal_flip=True, 
                  vertical_flip=True, 
                  rescale=None,
                  preprocessing_function=None,
                  data_format=None,
                  validation_split=0.0)
  datagen.fit(X_train)
  history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),  
                  epochs=epochs, 
                  steps_per_epoch=X_train.shape[0]//batch_size, 
                  validation_data=(X_test, Y_test), 
                  workers=10,
                  callbacks=callbacks)

## 2.4 Model preservation

In [ ]:
import os
from keras.models import Model

In [ ]:
model_path = os.path.join(save_dir, "classification_model.h5")
model.save(model_path)
model.summary()
print('Classification model saved at %s ' % model_path)

## 2.5 Training curves

In [ ]:
import os
import matplotlib.pyplot as plt

In [ ]:
# Plot Accuracy-Epoch 
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train set', 'Validation set'], loc='upper left')
plt.savefig(os.path.join(save_dir, 'model_accuracy.png'))
plt.show()

In [ ]:
# Plot Loss-Epoch
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train set', 'Validation set'], loc='upper left')
plt.savefig(os.path.join(save_dir, 'model_loss.png'))
plt.show()

## 2.6 Model prediction

In [ ]:
from keras.models import Model

In [ ]:
Y_test_pred=model.predict(X_test)
print(Y_test_pred)

## 2.7 Confusion matrix

In [ ]:
import itertools
import numpy as np
from sklearn.metrics import confusion_matrix

### 2.7.1 Prediction results on test set

In [ ]:
Y_test_true_label = []
for i in range(len(Y_test)):
  n = np.argmax(Y_test[i])
  Y_test_true_label.append(n)

Y_test_pred_label = []
for i in range(len(Y_test_pred)):
  n = np.argmax(Y_test_pred[i])
  Y_test_pred_label.append(n)

In [ ]:
print(Y_test_true_label)
print(Y_test)
print(Y_test_pred_label)
print(Y_test_pred)

### 2.7.2 [A] Confusion matrix

In [ ]:
def plot_confusion_matrix(cm, target_names, title='Confusion matrix', cmap=plt.cm.Greens, normalize=True):    
  accuracy = np.trace(cm) / float(np.sum(cm))
  misclass = 1 - accuracy
  if cmap is None:
    cmap = plt.get_cmap('Blues')
  plt.figure(figsize=(30, 30))   # (15,12)
  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  if target_names is not None:
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)
  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
  thresh = cm.max() / 1.5 if normalize else cm.max() / 2
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if normalize:
      plt.text(j, i, "{:0.4f}".format(cm[i, j]), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    else:
      plt.text(j, i, "{:,}".format(cm[i, j]), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
  plt.savefig(os.path.join(save_dir, "confusion_matrixA.png"), dpi=350)
  plt.show()

In [ ]:
 def plot_conf(y_pre, y_val, labels):
  conf_mat = confusion_matrix(y_true=y_val, y_pred=y_pre)
  print(conf_mat)
  plot_confusion_matrix(conf_mat, normalize=False, target_names=labels, title='Confusion Matrix')

In [ ]:
C = confusion_matrix(Y_test_true_label, Y_test_pred_label)
plot_conf(Y_test_pred_label, Y_test_true_label, CLASS)

### 2.7.2 [B] Confusion matrix

In [ ]:
def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.jet):
  cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  
  if cmap is None:
    cmap = plt.get_cmap('Blues')
  plt.figure(figsize=(15, 12))   # (15,12)   
  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, '{:.2f}'.format(cm[i, j]), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.savefig(os.path.join(save_dir, "confusion_matrixB.png"), dpi=350)
  plt.show()

In [ ]:
def plot_confuse(model, x_val, y_val):
  # predictions = model.predict_classes(x_val)
  predictions = np.argmax(model.predict(x_val),axis=1)
  truelabel = y_val.argmax(axis=-1) 
  conf_mat = confusion_matrix(y_true=truelabel, y_pred=predictions)
  plt.figure()
  plot_confusion_matrix(conf_mat, range(np.max(truelabel)+1))

In [ ]:
print(X_test.shape)
print(Y_test.shape)
plot_confuse(model, X_test, Y_test)

## 2.8 Feature maps

In [ ]:
import os
import keras
import numpy as np

In [ ]:
visual_folder = os.path.join(save_dir, "classification_model_visual")
visual_plot_folder = os.path.join(save_dir, "classification_model_visual_plot")

if not os.path.isdir(visual_folder):
  os.makedirs(visual_folder)

if not os.path.isdir(visual_plot_folder):
  os.makedirs(visual_plot_folder)

In [ ]:
def visual(model, data, num_layer):
  data = np.expand_dims(data, axis=0) 
  layer = keras.backend.function([model.layers[0].input], [model.layers[num_layer].output])
  f1 = layer([data])[0]
  num = f1.shape[-1]
  plt.figure(figsize=(8, 8))
  print("saving images in layer_"+str(num_layer)+" ...")
  for i in range(num):
    plt.subplot(np.ceil(np.sqrt(num)), np.ceil(np.sqrt(num)), i+1)
    layer_img = f1[0, :, :, i] * 255
    plt.imshow(layer_img, cmap='gray')
    plt.axis('off')
    visual_single_name = "layer_"+str(num_layer)+"_img_"+str(i)+".png"
    layer_single_path = os.path.join(visual_folder, visual_single_name)
    cv2.imwrite(layer_single_path, layer_img)
  layer_all_name = "layer_"+str(num_layer)+".png"
  layer_all_path = os.path.join(visual_plot_folder, layer_all_name)
  plt.savefig(layer_all_path, dpi=350)
  plt.show()

In [ ]:
for i in range(19): 
  visual(model, X_test[0], i)
print("All layer images saved!")